**Assessing building damage after a seismic event using unsupervised change detection with Sentinel-1 imagery**

Author: Augustin Martinet

**For the fulfillment of the degree of Master in Geographic Sciences, Geomatics Orientation, professional focus in geodata expert**

This work studies an unsupervised and multitemporal change detection method using Sentinel-1 imagery to assess buildings damage after the 2023 seismic event in Turkey.

A time series of the mean backscatter intensity from return signal
within the footprint of each destroyed building is evaluated to assess the change of trend after event.

Then, an omnibus test is used to assess changes across the time series: polarimetric SAR data are represented by complex variance-covariance matrices, whose homogeneity is evaluated through a factorization of multiple independent likelihood ratio tests in the time series. To determine where and when a change has occurred in our time series, we implemented a sequential omnibus tests algorithm assessing the overall differences between distributions at multiple times.

Input : AOI and footprints datasets - shapefiles
        Sentinel-1 Image Collection



## Import all the necessary libraries

In [58]:
import ee
import os
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library
ee.Initialize(project='ee-augustinmartinet')

In [59]:
import subprocess
import geemap
from google.colab import files
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline
from datetime import datetime
!pip install pycrs


To make use of interactive graphics, we import the _folium_ package:

In [60]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds Earth Engine layers to a folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True).add_to(self)
def add_ee_vector_layer(self, ee_object, vis_params, name):

        # Attempt to create a GeoJSON link from the Earth Engine object
        ee_data = ee_object.style(**vis_params).getMapId()
        folium.raster_layers.TileLayer(
            tiles=ee_data['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            name=name,
            overlay=True,
            control=True
        ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
folium.Map.add_ee_vector_layer = add_ee_vector_layer

# **Intensity temporal signatures of Destroyed building**

This Section studies the backscattered intensity over time from SAR signal for each building footprint. We built an interactive chart to visualize and examine the trend of each Destroyed building signature before/after event, for each orbit relativ and polarization.

## **1) Dataset & parameters definition**

### AOIs
Import shapefile of our 4 Area of Interest

In [ ]:
# Area of interest setting
AOI = geemap.shp_to_ee('/content/AOIs.shp')
center = AOI.filter(ee.Filter.eq('area_id', '04'))
aoi = AOI.geometry()

# Define center coordinates for map visualisation
coords = aoi.coordinates().getInfo()
geoJSON = {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": coords}}]}
corrected_coords = [polygon[0] for polygon in coords]

center = ee.Geometry.MultiPolygon(corrected_coords)
location = center.centroid().coordinates().getInfo()[::-1]

### Import building footprints dataset
Import shapefiles of 375 Destroyed and 375 Non-Destroyed building footprints with 20m buffers

In [ ]:
# Import Destroyed building footprints & buffers
OSM_Destroyed = geemap.shp_to_ee('/content/OSM_Destroyed.shp')
OSM_D_buf20 = geemap.shp_to_ee('/content/OSM_D_buf20.shp')
OSM_1_geom = OSM_D_buf20.geometry()

# Import Non-Destroyed building footprints & buffers
OSM_NDestroyed = geemap.shp_to_ee('/content/OSM_NDestroyed.shp')
OSM_ND_buf20 = geemap.shp_to_ee('/content/OSM_ND_buf20.shp')

### Set parameters
For now we set `parameters['Polarization']:'VV'`

Disaster event date is 06/02/2023

In [ ]:
parameters = {
    'Area': aoi,
    # 'CRS': 'EPSG:32650',
    'Polarization':'VV'}

geometry = parameters['Area']

# Define the event date
event_date = pd.to_datetime('2023-02-06')

## **2) Sentinel-1 Image collection**

### Descending and Ascending
We use Sentinel-1 ground range detected images.

Image collections are created for descending and ascending orbits, filtered by orbit pass, date range, and bounds, then sorted by date.


In [ ]:
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT'))

# S1 Collection sorted by date according to parameters
sentinel1_DESC = (sentinel1
          .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
           .filterDate(ee.Date('2022-10-01'),ee.Date('2023-03-02'))
           .filterBounds(aoi)
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

sentinel1_ASC = (sentinel1
          .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
           .filterDate(ee.Date('2022-10-01'),ee.Date('2023-03-13'))
           .filterBounds(aoi)
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

#### Check properties of first image

In [ ]:
#Properties
first_image = ee.Image(sentinel1_ASC.first())
properties = first_image.getInfo()['properties']

# Print the properties
#for property_name, property_value in properties.items():
    #print(f"{property_name}: {property_value}")

### Relative Orbits
The relative orbit numbers `relorb_DESC` and `relorb_ASC` for the descending and ascending image collections are combined into a single list `Relorb` as we will further process sorting by relative orbit.

In [ ]:
# Lists of Relativ Orbit combined into a single list
relorb_DESC = sentinel1_DESC.aggregate_histogram('relativeOrbitNumber_start')
relorb_DESC = [int(float(orbit_number)) for orbit_number in relorb_DESC.getInfo().keys()]


relorb_ASC = sentinel1_ASC.aggregate_histogram('relativeOrbitNumber_start')
relorb_ASC = [int(float(orbit_number)) for orbit_number in relorb_ASC.getInfo().keys()]

Relorb =  relorb_DESC + relorb_ASC
Relorb

[123, 21, 116, 14]

### Imprint
We observe the imprint of our image collections along with the AOIs and buildings dataset. Each AOI is covered by 6 relative orbits.

In [ ]:
Map = geemap.Map()

# Display imprint (median as composite layers) for each relative orbit in ASC and DESC
for orbit in relorb_ASC:
    ASC_by_orbit = sentinel1_ASC.filter(ee.Filter.eq('relativeOrbitNumber_start', orbit))
    asc_composite = ASC_by_orbit.median()
    Map.addLayer(asc_composite, {'min': 0, 'max': 0.3, 'bands': parameters['Polarization']}, f'ASC Orbit {orbit}')
    print(f'ASC collection for Orbit {orbit}: {ASC_by_orbit.size().getInfo()} images')
for orbit in relorb_DESC:
    DESC_by_orbit = sentinel1_DESC.filter(ee.Filter.eq('relativeOrbitNumber_start', orbit))
    desc_composite = DESC_by_orbit.median()
    Map.addLayer(desc_composite, {'min': 0, 'max': 0.3, 'bands': parameters['Polarization']}, f'DESC Orbit {orbit}')
    print(f'DESC collection for Orbit {orbit}: {DESC_by_orbit.size().getInfo()} images')

Map.addLayer(OSM_ND_buf20,{'color': '90EE90'}, "OSM_ND_buf20")
Map.addLayer(OSM_D_buf20,{'color': '006400'}, "OSM_D_buf20")
Map.addLayer(AOI, {'color': 'red'}, "AOIs")
Map.centerObject(aoi, 8)
Map

ASC collection for Orbit 116: 13 images
ASC collection for Orbit 14: 26 images
DESC collection for Orbit 123: 13 images
DESC collection for Orbit 21: 26 images


Map(center=[37.586844392530345, 37.6837696617634], controls=(WidgetControl(options=['position', 'transparent_b…

#### Convert to dB
Function to convert the specified polarization band of an image to decibel units and add it back to the original image as a new band. The dB scale is logarithmic, meaning it can better represent the wide range of radar backscatter intensities

In [ ]:
def to_db(img):
    """ Convert the specified polarization band to dB """
    polarization = parameters['Polarization']
    pol_band = img.select([polarization])
    pol_db = pol_band.log10().multiply(10).rename([f"{polarization}_db"])
    return img.addBands(pol_db)

## **3) Buildings Signatures dataframe**

#### Time series for a geometry
Functions to extract the time series of mean values for a given geometry (the building footprints) . When the geometry includes parts of multiple pixels, `reduceRegion` will weight the contribution of each pixel by the fraction of the pixel area covered by the geometry.


In [ ]:
def get_time_series(collection, geometry):
    """ Extract time series of mean value for a geometry """
    polarization = parameters['Polarization']
    pol_band = f'{polarization}_db'

    def extract_mean(image):
        """ Extract mean value for a geometry """
        date = image.date().format('YYYY-MM-dd')
        mean = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry).get(pol_band)
        return ee.Feature(None, {'date': date, 'mean': mean})

    time_series_means = collection.map(extract_mean).filter(ee.Filter.notNull(['mean']))
    means_list = time_series_means.getInfo()['features']

    # Convert the features to a list of dictionaries
    means_dicts = []
    for f in means_list:
        means_dict = {'date': f['properties']['date'], 'mean': f['properties']['mean']}
        means_dicts.append(means_dict)
    return means_dicts

#### Dataframe of time series
Function to create a dataframe of the time series **for each Destroyed building (OSM ID), orbit pass, relative orbit and polarization** .
Here we filter for computation purpose

In [ ]:
def fc_to_df(fc, collection, relative_orbit):
    """ Converts a feature collection to a dataframe with time series data for each feature"""
    orbit_type = collection.first().get('orbitProperties_pass').getInfo()
    data_frame = []

    # Append time series for each feature's geometry
    for index, feature in enumerate(fc.getInfo()['features'][:20]): # to filter
        geometry = ee.Geometry(feature['geometry'])
        osm_id = feature['properties']['OSM_ID']
        time_series = get_time_series(collection, geometry)

        if time_series:
            df = pd.DataFrame(time_series)
            df['date'] = pd.to_datetime(df['date'])
            df['OSM_ID'] = osm_id
            df['Orbit_Type'] = orbit_type
            df['Relative_Orbit'] = relative_orbit
            df['Polarization'] = parameters['Polarization']
            data_frame.append(df)
        else:
            continue

    if data_frame:
        return pd.concat(data_frame)
    else:
        return pd.DataFrame()

#### Dataframe by orbit
Function to create a dictionnary sorting the dataframe by relative orbit number, where each key is the orbit number.

In [ ]:
def df_by_orbit(collection, geometry, relative_orbit):
    """ Create a dictionnary of df for each orbit """
    data_by_orbit = {}
    for orbit_number in relative_orbit:
        collection_orbit = collection.filter(ee.Filter.eq('relativeOrbitNumber_start', orbit_number))

        if collection_orbit.size().getInfo() > 0:
            df = fc_to_df(geometry, collection_orbit, orbit_number)
            data_by_orbit[orbit_number] = {'df': df}
        else:
            continue

    return data_by_orbit

#### Process
Now we can process the above functions.  `df_by_orbit` will call the nested functions and create the dictionnaries of dataframes for each relative orbit.

Although `parameters['Polarization']` was already set for 'VV', it is redefined at entry .



In [ ]:
# Process VV polarization
parameters['Polarization'] = 'VV'
sentinel1_DESC_VV = sentinel1_DESC.map(to_db)
sentinel1_ASC_VV = sentinel1_ASC.map(to_db)

VV_DESC = df_by_orbit(sentinel1_DESC_VV, OSM_Destroyed, relorb_DESC)
VV_ASC = df_by_orbit(sentinel1_ASC_VV, OSM_Destroyed, relorb_ASC)

# Process VH polarization
parameters['Polarization'] = 'VH'
sentinel1_DESC_VH = sentinel1_DESC.map(to_db)
sentinel1_ASC_VH = sentinel1_ASC.map(to_db)

VH_DESC = df_by_orbit(sentinel1_DESC_VH, OSM_Destroyed, relorb_DESC)
VH_ASC = df_by_orbit(sentinel1_ASC_VH, OSM_Destroyed, relorb_ASC)

#### Concatenate DF
Each dataframes of the dictionnaries are merged back in one, **sorted by relative orbit and polarization.**

In [ ]:
# Concatenate dataframes from DESC
df_DESC_VV = [data['df'] for data in VV_DESC.values() if not data['df'].empty]
df_DESC_VH = [data['df'] for data in VH_DESC.values() if not data['df'].empty]

# Concatenate dataframes from ASC
df_ASC_VV = [data['df'] for data in VV_ASC.values() if not data['df'].empty]
df_ASC_VH = [data['df'] for data in VH_ASC.values() if not data['df'].empty]

# Concatenate both into our final dataframe
df_all = pd.concat(df_DESC_VV + df_ASC_VV + df_DESC_VH + df_ASC_VH, ignore_index=True)

#### Prints for check
For each of the 375 Destroyed buildings, there is two polarizations with three relative orbit, for a total of 2250 signatures. Over our timeframe of 13 dates, it will give a dataframe of 29 250 values, the mean intensity over the footprint geometry at instant t.

In [ ]:
print(df_all.head())
num_points = len(df_all)
print(f"Nombre de points dans le DataFrame : {num_points}")

        date      mean    OSM_ID  Orbit_Type  Relative_Orbit Polarization
0 2022-10-08 -0.452102  91668879  DESCENDING             123           VV
1 2022-10-20  1.367219  91668879  DESCENDING             123           VV
2 2022-11-01 -0.613894  91668879  DESCENDING             123           VV
3 2022-11-13 -1.074031  91668879  DESCENDING             123           VV
4 2022-11-25 -0.295816  91668879  DESCENDING             123           VV
Nombre de points dans le DataFrame : 1560


In [ ]:
for i, df in enumerate(df_DESC_VV):
    print(f"df_DESC_VV[{i}]:")
    print(df.head())
    print("\n")

for i, df in enumerate(df_DESC_VH):
    print(f"df_DESC_VH[{i}]:")
    print(df.head())
    print("\n")

for i, df in enumerate(df_ASC_VV):
    print(f"df_ASC_VV[{i}]:")
    print(df.head())
    print("\n")

for i, df in enumerate(df_ASC_VH):
    print(f"df_ASC_VH[{i}]:")
    print(df.head())
    print("\n")

df_DESC_VV[0]:
        date      mean    OSM_ID  Orbit_Type  Relative_Orbit Polarization
0 2022-10-08 -0.452102  91668879  DESCENDING             123           VV
1 2022-10-20  1.367219  91668879  DESCENDING             123           VV
2 2022-11-01 -0.613894  91668879  DESCENDING             123           VV
3 2022-11-13 -1.074031  91668879  DESCENDING             123           VV
4 2022-11-25 -0.295816  91668879  DESCENDING             123           VV


df_DESC_VV[1]:
        date      mean    OSM_ID  Orbit_Type  Relative_Orbit Polarization
0 2022-10-01 -1.906672  91668879  DESCENDING              21           VV
1 2022-10-13 -0.155776  91668879  DESCENDING              21           VV
2 2022-10-25 -2.089277  91668879  DESCENDING              21           VV
3 2022-11-06 -1.488518  91668879  DESCENDING              21           VV
4 2022-11-18 -0.684852  91668879  DESCENDING              21           VV


df_DESC_VH[0]:
        date      mean    OSM_ID  Orbit_Type  Relative_Orbit Po

## **Interactive chart of Destroyed buildings signatures**


Here is the final tools of **Section I**:

An interactive chart displaying **the mean intensity for each building footprint over time, with distinct polarization types, relative orbits**. It allows to visualize and examine the trend of each signature before/after event.

#### Create chart

In [ ]:
def create_combined_chart(df):
    """Generate an interactive Altair chart visualizing backscatter data """

    df['ID_Orbit_Combination'] = df['OSM_ID'].astype(str) + ' | ' + df['Orbit_Type'] + ' | ' + df['Relative_Orbit'].astype(str) + ' | ' + df['Polarization']

    df['Orbit_Combination'] = df['Orbit_Type'] + ' | ' + df['Relative_Orbit'].astype(str) + ' | ' + df['Polarization']

    highlight = alt.selection(type='single', on='mouseover', fields=['OSM_ID'], nearest=True, empty='none')

    vertical_line = alt.Chart(pd.DataFrame({'date': [pd.Timestamp('2023-02-06')]})).mark_rule(
        color='black', strokeWidth=1
    ).encode(x='date:T')

    # Base configuration of the chart
    base = alt.Chart(df).encode(
        x=alt.X('date:T', title='Date'),
        y=alt.Y('mean:Q', title='Backscatter (dB)', scale=alt.Scale(zero=False)),
        color=alt.Color('Relative_Orbit:N', legend=alt.Legend(title="Relative Orbit")),
        detail='OSM_ID:N',
        tooltip=[
            alt.Tooltip('ID_Orbit_Combination:N', title='Combination'),
            alt.Tooltip('date:T', title='Date', format='%Y-%m-%d'),
            alt.Tooltip('mean:Q', title='Backscatter'),
            alt.Tooltip('Polarization:N', title='Polarization')
        ]
    )

    points = base.mark_circle(size=60).encode(opacity=alt.value(0)).add_selection(highlight)

    lines = base.mark_line().encode(
        strokeDash=alt.StrokeDash('Polarization:N', legend=alt.Legend(title="Orbit Combination"),
                                   scale=alt.Scale(domain=['VV', 'VH'], range=[[], [5, 5]])),
        size=alt.condition(highlight, alt.value(3), alt.value(1.5)),
        opacity=alt.condition(highlight, alt.value(1), alt.value(0.1))
    )
    chart = (points + lines + vertical_line).properties(width=1000, height=500).interactive()

    return chart

#### Chart Vizualisation Tools
Memory allows to display only 5000 values. The whole dataset is available on the complementary **Observable HQ notebook :** https://observablehq.com/@ulgeovisualization/damage-assesment-using-sar

In [ ]:
#  df_sample or df_all
df_sample = df_all.sort_values(by='OSM_ID').head(5000)
chart_combined = create_combined_chart(df_sample)
chart_combined

alt.LayerChart(...)

#### Filter Relative orbit
We can program some different options, e.g. to observe only one relative orbit. The Observable HQ Notebook will be more suitable for data visualization and such options filtering

In [ ]:
######## Here selection of relative orbit to display ############
relorb_to_display = 21
#################################################################

df_sample2 = df_all[df_all['Relative_Orbit'] == relorb_to_display]

# Get the first 150 unique OSM_ID values
unique_osm_ids = df_sample2['OSM_ID'].unique()[:150]

# Filter the DataFrame to include only these OSM_ID values
df_filtered = df_sample2[df_sample2['OSM_ID'].isin(unique_osm_ids)]

points_filtered = len(df_filtered)
print(f"Nombre de points dans le DataFrame : {points_filtered}")

chart_2 = create_combined_chart(df_filtered)
chart_2

Nombre de points dans le DataFrame : 520


alt.LayerChart(...)

## **4) Z-Score**

From the time series signatures obtained, we perform a z-score standardization to identify and observe deviations in building signatures after the event compared to the mean of values before

#### Calculate z-scores
First, split the dataframe in before and after event date and measure the mean of each part, then measure the stantard deviation for the before event part.
At last, measures the Z-score, as the difference in means after the event compared to before, relatively to the variability before the event.


In [ ]:
def calculate_means(df, event_date):
    """ Function to measure the mean of values before and after the event """
    # Split the data into before and after the event
    df_before = df[df['date'] < event_date]
    df_after = df[df['date'] >= event_date]

    # Calculate mean for each OSM_ID, Relative_Orbit, and Polarization before the event
    means_before = df_before.groupby(['OSM_ID', 'Relative_Orbit', 'Polarization'])['mean'].mean().reset_index()
    means_before.rename(columns={'mean': 'mean_before'}, inplace=True)

    # Calculate mean for each OSM_ID, Relative_Orbit, and Polarization after the event
    means_after = df_after.groupby(['OSM_ID', 'Relative_Orbit', 'Polarization'])['mean'].mean().reset_index()
    means_after.rename(columns={'mean': 'mean_after'}, inplace=True)

    # Merge the before and after means on OSM_ID, Relative_Orbit, and Polarization
    means_combined = pd.merge(means_before, means_after, on=['OSM_ID', 'Relative_Orbit', 'Polarization'], how='outer')

    return means_combined

def calculate_stats(df, event_date):
    """ Function to calculate mean, standard deviation for each OSM_ID, Relative_Orbit, and Polarization """
    # Split the data into before and after the event
    df_before = df[df['date'] < event_date]
    df_after = df[df['date'] >= event_date]

    # Calculate mean and standard deviation for each OSM_ID, Relative_Orbit, and Polarization before the event
    stats_before = df_before.groupby(['OSM_ID', 'Relative_Orbit', 'Polarization'])['mean'].agg(['mean', 'std']).reset_index()
    stats_before.rename(columns={'mean': 'mean_before', 'std': 'std_before'}, inplace=True)

    # Calculate mean for each OSM_ID, Relative_Orbit, and Polarization after the event
    means_after = df_after.groupby(['OSM_ID', 'Relative_Orbit', 'Polarization'])['mean'].mean().reset_index()
    means_after.rename(columns={'mean': 'mean_after'}, inplace=True)

    # Merge the before and after stats on OSM_ID, Relative_Orbit, and Polarization
    stats_combined = pd.merge(stats_before, means_after, on=['OSM_ID', 'Relative_Orbit', 'Polarization'], how='outer')

    return stats_combined

def calculate_z_scores(df):
    """ Function to calculate z-scores """
    df['z_score'] = (df['mean_after'] - df['mean_before']) / df['std_before']
    return df

#### Results overview can be displayed in a Table here

In [ ]:
# Apply the functions to the dataFrame
stats_combined = calculate_stats(df_all, event_date)
stats_with_z_scores = calculate_z_scores(stats_combined)

# Drop columns that are no longer needed
stats_with_z_scores = stats_with_z_scores.drop(columns=['mean_diff', 'std_dev'], errors='ignore')
df_z_scores = stats_with_z_scores.drop(columns=['mean_diff', 'std_dev', 'mean_before',	'std_before',	'mean_after'], errors='ignore')
# Display the dataFrame
stats_with_z_scores.head()

,OSM_ID,Relative_Orbit,Polarization,mean_before,std_before,mean_after,z_score
0,222765255,21,VH,-6.565970,0.587134,-11.613270,-8.596499
1,222765255,21,VV,6.352165,0.808273,-2.404840,-10.834223
2,222765255,116,VH,-12.784958,0.681588,-10.720561,3.028805
3,222765255,116,VV,0.381238,1.239243,-5.527042,-4.767655
4,222765255,123,VH,-3.949027,1.513251,-12.066668,-5.364372


In [ ]:
# Define the range bounds
bounds = {
    'between -1 and 1': (-1, 1),
    'between -2 and 2': (-2, 2),
    'between -3 and 3': (-3, 3)
}

# Function to check OSM_IDs based on bounds
def get_osmids_within_bounds(df, lower_bound, upper_bound):
    return df.groupby('OSM_ID').apply(lambda x: ((x['z_score'] >= lower_bound) & (x['z_score'] <= upper_bound)).all())

# Loop through the bounds and check OSM_IDs for each case
for key, (lower_bound, upper_bound) in bounds.items():
    osmids_within_bounds = get_osmids_within_bounds(df_z_scores, lower_bound, upper_bound)
    osmids_list = osmids_within_bounds[osmids_within_bounds].index
    print(f"OSM_IDs with all signatures {key}: {list(osmids_list)}")
    print(f"Number of OSM_IDs with all signatures {key}: {len(osmids_list)}\n")


OSM_IDs with all signatures between -1 and 1: []
Number of OSM_IDs with all signatures between -1 and 1: 0

OSM_IDs with all signatures between -2 and 2: ['392441117', '392441118']
Number of OSM_IDs with all signatures between -2 and 2: 2

OSM_IDs with all signatures between -3 and 3: ['351353004', '373392020', '389615296', '392441117', '392441118']
Number of OSM_IDs with all signatures between -3 and 3: 5



#### Summarize results
To summarize results we get a count for scores above or below 1, 2 or 3 sigma

In [ ]:
# Counting the number of signatures above or below certain z-score thresholds
thresholds = [1, 2, 3]
results = {}

for threshold in thresholds:
    count_above = (stats_with_z_scores['z_score'] > threshold).sum()
    count_below = (stats_with_z_scores['z_score'] < -threshold).sum()
    count_between = ((stats_with_z_scores['z_score'] <= threshold) & (stats_with_z_scores['z_score'] >= -threshold)).sum()
    results[threshold] = {'above': count_above, 'below': count_below, 'between': count_between}

results

{1: {'above': 15, 'below': 75, 'between': 30},
 2: {'above': 6, 'below': 50, 'between': 64},
 3: {'above': 2, 'below': 34, 'between': 84}}

#### Export

In [ ]:
num_points = len(df_all)
print(f"Nb of points in DF : {num_points}")


Nb of points in DF : 1560


In [ ]:
""" Export dataframe"""

# Merge the z-scores back into the original DataFrame
df_final = pd.merge(df_all, stats_with_z_scores[['OSM_ID', 'Relative_Orbit', 'Polarization', 'z_score']],
                            on=['OSM_ID', 'Relative_Orbit', 'Polarization'], how='left')

# Drop columns that are no longer needed
df_final = df_final.drop(columns=['mean_before', 'std_before', 'mean_after'], errors='ignore')

df_final['z_score'] = df_final['z_score'].round(2)
df_final['mean'] = df_final['mean'].round(2)

# Reorder and select the desired columns
desired_columns = ['date', 'OSM_ID', 'mean', 'Orbit_Type', 'Relative_Orbit', 'Polarization', 'z_score']
df_final = df_final[desired_columns]

# Display the modified DataFrame
print(df_final.head())

# Export the final DataFrame to a CSV file
output_file = 'final_df.csv'
df_final.to_csv(output_file, index=False)
print(f"DataFrame exported to {output_file}")
#files.download(output_file)

        date    OSM_ID  mean  Orbit_Type  Relative_Orbit Polarization  z_score
0 2022-10-08  91668879 -0.45  DESCENDING             123           VV    -1.91
1 2022-10-20  91668879  1.37  DESCENDING             123           VV    -1.91
2 2022-11-01  91668879 -0.61  DESCENDING             123           VV    -1.91
3 2022-11-13  91668879 -1.07  DESCENDING             123           VV    -1.91
4 2022-11-25  91668879 -0.30  DESCENDING             123           VV    -1.91
DataFrame exported to final_df.csv


# **Multitemporal change detection**

This section is inspired from the GEE tutorial on Detecting Changes in Sentinel-1 imagery [Part 3](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3) to build multitemporal change maps applied to our case.

## **1) Merged Images lists**

It will turn out to be convenient to work with a list rather than a collection, so we'll convert our collections to lists. However, our collections for orbit 21 and 14 cover two imprints so they need to be merged in order to make one list with unique dates.

### Timestamp


We retrieve the acquisition dates from each image in the collections as timestamps to merge those taken at same date

In [ ]:
timestampDESC = (sentinel1_DESC.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())

timestampASC = (sentinel1_ASC.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())
timestampDESC
#timestampASC

['T20221001',
 'T20221001',
 'T20221008',
 'T20221013',
 'T20221013',
 'T20221020',
 'T20221025',
 'T20221025',
 'T20221101',
 'T20221106',
 'T20221106',
 'T20221113',
 'T20221118',
 'T20221118',
 'T20221125',
 'T20221130',
 'T20221130',
 'T20221207',
 'T20221212',
 'T20221212',
 'T20221219',
 'T20221224',
 'T20221224',
 'T20221231',
 'T20230105',
 'T20230105',
 'T20230112',
 'T20230117',
 'T20230117',
 'T20230124',
 'T20230129',
 'T20230129',
 'T20230205',
 'T20230210',
 'T20230210',
 'T20230217',
 'T20230222',
 'T20230222',
 'T20230301']

### Merging images by date
Functions to merge images by orbit number and date

In [ ]:
def merge_by_date(images, date):
    """Merge images by date """
    merged_image = images.filter(ee.Filter.eq('date', date)).mosaic().set('date', date)
    return merged_image

def merge_to_coll(sentinel_collection, orbit_numbers):
    """ Process merge by orbit number and date"""
    merged_collections = {}
    for orbit_number in orbit_numbers:
        images_by_orbit = sentinel_collection.filter(ee.Filter.eq('relativeOrbitNumber_start', orbit_number))
        unique_dates = images_by_orbit.aggregate_array('date').distinct().getInfo()
        merged_list = []
        for date in unique_dates:
            merged_image = merge_by_date(images_by_orbit, date)
            merged_list.append(merged_image)
        merged_collections[orbit_number] = ee.ImageCollection(merged_list)
    return merged_collections

### List definition

We obtain dictionaries of merged images lists with the relative orbit as index.

 **Here**, we select the `im_list` from orbit we want to work on (by uncomment).

In [ ]:
# Process ASC and DESC collections by orbit number
desc_by_orbit = merge_to_coll(sentinel1_DESC, relorb_DESC)
asc_by_orbit = merge_to_coll(sentinel1_ASC, relorb_ASC)

#im_list = desc_by_orbit[123].toList(desc_by_orbit[123].size())
im_list = desc_by_orbit[21].toList(desc_by_orbit[21].size())
#im_list = asc_by_orbit[116].toList(asc_by_orbit[116].size())
#im_list = asc_by_orbit[14].toList(asc_by_orbit[14].size())

im_list.length().getInfo()


13

In [ ]:
im_list

### Clip images
We clip the images to our AOIs concerned ( or to the building footprints)..

In [ ]:
def clip_img(img):
    """Clip a list of images."""
    return ee.Image(img).clip(aoi)
im_list = ee.List(im_list.map(clip_img))

## **2 ) An omnibus test for change**

From here, the method and theory follows the GEE tutorial on Detecting Changes in Sentinel-1 imagery [Part 3](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3).

The idea is to calculate the omnibus test statistic for each pixel's intensity up to a given time and compare the frequency of the values obtained to a statistical distribution.

### Select polarization

Initially, we select a polarization from our `im_list` to start with the easier single polarization and use this `vv_list` for now.

In [ ]:
def selectpol(current, polarization):
    """Selects the specified polarization band of the given image."""
    return ee.Image(current).select(polarization)

# Create lists for VV and VH polarizations
vv_list = im_list.map(lambda img: selectpol(img, 'VV'))
vh_list = im_list.map(lambda img: selectpol(img, 'VH'))


For visualization, here is an RGB composite of the VV bands for three images before event  

In [ ]:
mp = folium.Map(location=location, zoom_start=11)
Map.addLayer(OSM_ND_buf20,{'color': '90EE90'}, "OSM_ND_buf20")
Map.addLayer(OSM_D_buf20,{'color': '006400'}, "OSM_D_buf20")
Map.addLayer(OSM_ND_buf20,{'color': '90EE90'}, "OSM_ND_buf20")
Map.addLayer(OSM_D_buf20,{'color': '006400'}, "OSM_D_buf20")
rgb_images = (ee.Image.rgb(vv_list.get(8), vv_list.get(9), vv_list.get(10)).log10().multiply(10))
mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
mp.add_child(folium.LayerControl())

### Theory

For the series of _VV_ intensity images acquired at times $t_1, t_2,\dots t_k$, our null hypothesis is that, at a given pixel position,  there has been no change in the signal strengths $a_i=\langle|S^{a_i}_{vv}|^2\rangle$ over the entire period, i.e.,

$$
H_0:\quad a_1 = a_2 = \dots = a_k = a.
$$

The alternative hypothesis is that there was at least one change (and possibly many) over the interval. For the more mathematically inclined this can be written succinctly as

$$
H_1:\quad \exists\ i,j :\ a_i \ne a_j,
$$

which says: there exist indices $i, j$ for which $a_i$ is not equal to $a_j$.

Again, the likelihood functions are products of gamma distributions:

$$
L_1(a_1,\dots,a_k) =\prod_{i=1}^k p(s_i\mid a_i) = {1\over\Gamma(m)^k}\left[\prod_i{a_i\over m}\right]^{-m}\left[\prod_i s_i\right]^{m-1}\exp(-m\sum_i{s_i\over a_i}) \tag{3.3}
$$
*where $ L_1(a_1,\dots,a_k) $ is — Likelihood under $H_1$ , optimized individually for each time point based on the observed data.
And $L_0(a)$ — Likelihood under $H_0$ , optimized for a common a across all time points.*


$$
L_0(a)  = \prod_{i=1}^k p(s_i\mid a) = {1\over\Gamma(m)^k} \left[{a\over m}\right]^{-mk}\left[\prod_i s_i\right]^{m-1}\exp(-{m\over a}\sum_i s_i) \tag{3.4}
$$

and $L_1$ is maximized for $\hat a_i = s_i,\ i=1\dots k,$ while $L_0$ is maximized for $\hat a = {1\over k}\sum_i s_i$. So with a bit of simple algebra our Likelihood Ratio Test statistic ($Q_k$) is :  *the ratio of $L_0$ to $L_1$, raised to the power m (number of looks) and then simplified using algorithmic transformations to handle the products in the likelihoods:*



$$
Q_k = {L_0(\hat a)\over L_1(\hat a_1,\dots,\hat a_k)} = \left[k^k{\prod_i s_i\over (\sum_i s_i)^k}\right]^m \tag{3.5}
$$

and is called an _omnibus test statistic_. Note that, for $k=2$, we get the bitemporal LRT given by  [Eq. (2.10)](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#the_likelihood_ratio_test).

*$Q_k$ is then converted to $-2log Q_k$ to facilitate comparison against a chi-square distribution, leveraging Wilks' theorem.*

We can't expect to find an analytical expression for the probability distribution of this LRT statistic, so we will again invoke Wilks' Theorem and work with

$$
-2 \log{Q_k} = \big[k\log{k}+\sum_i\log{s_i}-k\log{\sum_i s_i}\big](-2m) \tag{3.6}
$$

According to Wilks, it should be approximately chi square distributed with $k-1$ degrees of freedom under $H_0$. (Why?)
*The degrees of freedom are k-1 because the null hypothesis imposes one constraint across k observations.*

### Omnibus function

The input cell below evaluates the test statistic Eq. (3.6) for a list of single polarization images. We prefer from now on to use as default the equivalent number of looks 4.4 that is discussed at the end of [Part 1](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1#equivalent_number_of_looks) rather than the actual number of looks $m=5$, in the hope of getting a better agreement.

In [ ]:
def omnibus(im_list, m = 4.4):
    """Calculates the omnibus test statistic for a list, monovariate case."""
    def log(current):
        return ee.Image(current).log()

    im_list = ee.List(im_list)
    k = im_list.length()
    klogk = k.multiply(k.log())
    klogk = ee.Image.constant(klogk)
    sumlogs = ee.ImageCollection(im_list.map(log)).reduce(ee.Reducer.sum())
    logsum = ee.ImageCollection(im_list).reduce(ee.Reducer.sum()).log()
    return klogk.add(sumlogs).subtract(logsum.multiply(k)).multiply(-2*m)

### Chi square distribution

Let's see if this omnibus test statistic does indeed follow the chi square distribution under the null hypothesis of no change. We perform the test on both our buildings footprint datasets, Destroyed and Non-Destroyed, as well as the whole AOIs for reference. We do so for both before and after event. So we can expect one distribution with change and one without.

Here is first a comparison for pixels in both footprint datasets with the chi square distribution with $k-1$ degrees of freedom.We chose the first 10 images (before event), and the last 3 images (around event).

#### Histogram generation
Functions to compute a histogram of the omnibus test statistic values for a subset of images over a specified geometry.

In [ ]:
def get_histogram(vv_list, geometry, start, end, bins=200, range_min=0, range_max=40):
    """ Computes a histogram of the omnibus test statistic values"""
    hist = (omnibus(vv_list.slice(start, end))
            .reduceRegion(ee.Reducer.fixedHistogram(range_min, range_max, bins),
                          geometry=geometry, scale=10)
            .get('constant')
            .getInfo())
    return hist

 Frequencies of values get normalized to compare them with a theoretical chi-square distribution defined. We convert them into dataframes for using Altair charts visualization.

In [ ]:
def norm_histogram(hist, label, dof):
    """ Normalizes frequencies and returns dataframe """
    a = np.array(hist)
    x = a[:, 0]
    y = a[:, 1] / np.sum(a[:, 1])
    chi2_y = chi2.pdf(x, dof) / 5
    return pd.DataFrame({
        'x': x,
        'y': y,
        'chi2_y': chi2_y,
        'label': label
    })

We compile histograms for both footprints datasets and AOIs and both subsets of images: the first 10 images, which are all before the event and the last 3 images, which are one before event and two after.

In [ ]:
# List of geometries
geometries = [OSM_D_buf20, OSM_ND_buf20, aoi]
labels = ['Destroyed Buffer', 'Non-Destroyed Buffer', 'AOIs']

#Slices of images
# k-1 degrees of freedom for chi-squared
k = im_list.length().getInfo()
k_first = 10  # first 10 images
k_last = 3 # 3 last

# Combined DataFrame for all before histograms.
before_histo = pd.DataFrame()
for geometry, label in zip(geometries, labels):
    hist = get_histogram(vh_list, geometry,0 , k_first)
    df = norm_histogram(hist, label, dof=k_first -1)
    before_histo = pd.concat([before_histo, df], ignore_index=True)

# Combined DataFrame for all after histograms.
after_histo = pd.DataFrame()
for geometry, label in zip(geometries, labels):
    hist = get_histogram(vh_list, geometry, k - k_last, k)
    df = norm_histogram(hist, label, dof=k_last-1)
    after_histo = pd.concat([after_histo, df], ignore_index=True)

Function to create an interactive visualization to display all distributions of datasets separately.

In [ ]:
def create_histogram(df):
    """ Creates an interactive histogram using Altair """
    alt.data_transformers.disable_max_rows()
    highlight = alt.selection(type='single', on='mouseover', fields=['label'], nearest=True, empty='none')

    color_scale = alt.Scale(domain=['Destroyed Buffer', 'Non-Destroyed Buffer', 'AOIs'],
                            range=['green', 'purple', 'red'])

    points = alt.Chart(df).mark_circle(size=20).encode(
        x=alt.X('x:Q', bin=False, title='Value',
                axis=alt.Axis(titleFontSize=18)),
        y=alt.Y('y:Q', title='Normalized Frequency',
                axis=alt.Axis(titleFontSize=18)),
        color=alt.Color('label:N', scale=color_scale,
                        legend=alt.Legend(title="Dataset", titleFontSize=16, labelFontSize=14)),
        opacity=alt.condition(highlight, alt.value(1), alt.value(0.9))
    ).add_selection(highlight)

    line = alt.Chart(df).mark_line(color='black').encode(
        x='x:Q',
        y='chi2_y:Q'
    )

    return (points + line).properties(width=600)



#### Before event

In [ ]:
chart_before = create_histogram(before_histo)
chart_before

The histograms represent the distribution of the omnibus test statistic values over our subset of images.

The x-axis shows the range of omnibus test statistic values. The y-axis represents the frequency of these values.  

All datasets follow a chi-square distribution and are rather identical.

#### Around event

In [ ]:
chart_after = create_histogram(after_histo)
chart_after

Here again, the 'Non-Destroyed' and AOIs dataset follows closely the chi-square distribution, as expected for no change.

On the other hand for the 'Destroyed' dataset, we can observe some deviation from the theoretical distribution, with relatively less frequent lower values indicating less occurences of no change (values less than 4), and more values higher than 4 which indicates more changes.  

It appears that Wilks' Theorem is again a fairly good approximation. So why not generate a change map for the full series? The good news is that we now have the overall false positive probability α under control. Here we set it to α=0.25, then to others values for our **Sensitivity Analysis**.

### Change map for single polarization

In [ ]:
alpha = 0.01

A chi-squared cumulative distribution function (CDF) is used to compute p-values, to determine how extreme the test statistic is under the null hypothesis. The PDF gave the likelihood of a specific value of the test statistic occurring. Here, the CDF gives the probability (p-value) that the test statistic will take a value less than or equal to the observed value, which is then compared against the threshold significance level (α).

In [ ]:
def chi2cdf(chi2, df):
    """Calculates Chi square cumulative distribution function for
       df degrees of freedom using the built-in incomplete gamma
       function gammainc().
    """
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

Here, a change map is created where each pixel's p-value is checked against a significance level 𝛼=0.01. If the p-value is less than 0.01, it suggests significant change, and the pixel value is set to 1 (indicating change); otherwise, it remains 0 (no change). The Destroyed and Non-Destroyed building footprints are added in dark and light green respectively.

In [ ]:
p_value = ee.Image.constant(1).subtract(chi2cdf(omnibus(vv_list), k-1))
c_map = p_value.multiply(0).where(p_value.lt(alpha), 1)
c_map = c_map.updateMask(c_map.gt(0))

mp = folium.Map(location=location, zoom_start=11)
mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
mp.add_ee_vector_layer(OSM_ND_buf20, {'color': 'blue', 'width': 2, 'fillColor': '#00000000'},'OSM Non-Destroyed')
mp.add_ee_vector_layer(OSM_D_buf20, {'color': 'green', 'width': 2, 'fillColor': '#00000000'},'OSM Destroyed')
mp.add_ee_layer(c_map, {'min': 0,'max': 1, 'palette': ['black', 'red']}, 'change map')
mp.add_child(folium.LayerControl())

So plenty of changes, but hard to interpret considering the time span. Although we can see where changes took place, we know neither when they occurred nor their multiplicity.

## **3) A sequential omnibus test**

### Theory


Recalling the last remark at the end of [Part 2](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#oh_and_one_more_thing_), let's now guess the omnibus LRT for the dual polarization case. From Eq. (3.5), replacing $s_i \to|c_i|$,  $\ \sum s_i \to |\sum c_i|\ $ and $k^k \to k^{2k}$, we get

$$
Q_k =  \left[k^{2k}{\prod_i |c_i|\over |\sum_i c_i|^k}\right]^m. \tag{3.7}
$$

This is in fact a special case of a more general omnibus test statistic

$$
Q_k =  \left[k^{pk}{\prod_i |c_i|\over |\sum_i c_i|^k}\right]^m
$$

which holds for $p\times p$ polarimetric covariance matrix images, for example for the full dual pol matrix   [Eq. (1.5)](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1#single_look_complex_slc_sar_measurements) or for full $3\times 3$ quad pol matrices ($p=3$), but also for diagonal $2\times 2$ and $3\times 3$ matrices.

Which brings us to the **heart of this Tutorial**. We will now decompose Eq. (3.7) into a product of independent likelihood ratio tests which will enable us to determine when changes occurred at each pixel location. Then we'll code a complete multitemporal change detection algorithm on the GEE Python API.

#### Single polarization example



Rather than make a formal derivation, we will illustrate the decomposition on a series of $k=5$ single polarization (VV) measurements. The omnibus test Eq. (3.5) for any change over the series from $t_1$ to $t_5$ is

$$
Q_5 = \left[ 5^5 {s_1s_2s_3s_4s_5\over (s_1+s_2+s_3+s_4+s_5)^5}\right]^m.
$$

If we accept the null hypothesis $a_1=a_2=a_3=a_4=a_5$ we're done and can move on to the next pixel (figuratively of course, since this stuff is all done in parallel). But suppose we have rejected the null hypothesis, i.e., there was a least one significant change. In order to find it (or them), we begin by testing the first of the four intervals. That's just the bitemporal test from Part 2, but let's call it $R_2$ rather than $Q_2$,

$$
R_2 = \left[ 2^2 {s_1s_2\over (s_1+s_2)^2}\right]^m.
$$

Suppose we conclude no change, that is, $a_1=a_2$. Now we don't do just another bitemporal test on the second interval. Instead we test the hypothesis

$$
\begin{align*}
H_0:\ & a_1=a_2= a_3\ (=a)\cr
{\rm against}\quad H_1:\  &a_1=a_2\ (=a) \ne a_3.
\end{align*}
$$

So the alternative hypothesis is: _There was no change in the first interval **and** there was a change in the second interval_. The LRT is easy to derive, but let's go through it anyway.

$$
\begin{align*}
        {\rm From\ Eq.}\ (3.4):\  &L_0(a)  = {1\over\Gamma(m)^3} \left[{a\over m}\right]^{-3m}\left[s_1s_2s_3\right]^{m-1}\exp(-{m\over a}(s_1+s_2+s_3)  \cr
        &\hat a = {1\over 3}(s_1+s_2+s_3) \cr
=>\           &L_0(\hat a) = {1\over\Gamma(m)^3} \left[{s_1+s_2+s_3\over 3m}\right]^{-3m}\left[s_1s_2s_3\right]^{m-1} \exp(-3m) \cr
{\rm From\ Eq.}\ (3.3):\ &L_1(a_1,a_2,a_3) = {1\over\Gamma(m)^3}\left[a_1a_2a_3\over m\right]^{-m}[s_1s_2s_3]^{m-1}\exp(-m(s_1/a_1+s_2/a_2+s_3/a_3)\cr
&\hat a_1 = \hat a_2 = {1\over 2}(s_1+s_2),\quad \hat a_3 = s_3 \cr
=>\ &L_1(\hat a_1,\hat a_2, \hat a_3) = {1\over\Gamma(m)^3}\left[(s_1+s_2)^2s_3\over 2^2m \right]^{-m}[s_1s_2s_3]^{m-1}\exp(-3m)
\end{align*}
$$

And, taking the ratio $L_0/L_1$of the maximum likelihoods,

$$
R_3 = \left[{3^3\over 2^2}{(s_1+s_2)^2s_3\over (s_1+s_2+s_3)^3}\right]^m.
$$

Not too hard to guess that, if we accept $H_0$ again, we go on to test

$$
\begin{align*}
H_0:\ a_1=a_2=a_3=a_4\ (=a)\cr
{\rm against}\quad H_1:\ a_1=a_2=a_3\ (=a) \ne a_4.
\end{align*}
$$

with LRT statistic

$$
R_4 = \left[{4^4\over 3^3}{(s_1+s_2+s_3)^3s_4\over (s_1+s_2+s_3+s_4)^4}\right]^m,
$$

and so on to $R_5$ and the end of the time series.

Now for the cool part:

$$
R_2\times R_3\times R_4 \times R_5 = Q_5.
$$

So, generalizing to a series of length $k$:

**The omnibus test statistic $Q_k$ may be factored into the product of  LRT's $R_j$ which test for homogeneity in the measured reflectance signal up to and including time $t_j$, assuming homogeneity up to time $t_{j-1}$:**

$$
Q_k = \prod_{j=2}^k R_j, \quad R_j = \left[{j^j\over (j-1)^{j-1}}{(s_1+\dots +s_{j-1})^{j-1}s_j\over (s_1+\dots +s_j)^j}\right]^m,\quad j = 2\dots k.  \tag{3.8}
$$

Moreover the test statistics $R_j$ are stochastically independent under $H_0$.
This is shown analytically, see [Conradsen et al. (2016)](https://ieeexplore.ieee.org/document/7398022) or P. 405 in my [textbook](https://www.taylorfrancis.com/books/9780429464348), and empirically in the tutorial by sampling the test statistics $R_j$ in the test region and examining the correlation matrix.

#### Dual polarization and an algorithm


With our substitution trick, we can now write down the sequential test for the dual polarization (bivariate) image time series. From Eq. (3.8) we get

$$
Q_k = \prod_{j=2}^k R_j , \quad R_j = \left[{j^{2j}\over (j-1)^{2(j-1)}}{|c_1+\dots +c_{j-1}|^{j-1}|c_j|\over |c_1+\dots +c_j|^j}\right]^m,\quad j = 2\dots k. \tag{3.9}
$$

And of course we have again to use Wilks' Theorem to get the _P_ values, so we work with

$$
-2\log{R_j} = -2m\Big[2(j\log{j}-(j-1)\log(j-1)+(j-1)\log\Big|\sum_{i=1}^{j-1}c_i \Big|+\log|c_j|-j\log\Big|\sum_{i=1}^j c_i\Big|\ \Big] \tag{3.10a}
$$

and

$$
-2\log Q_k = \sum_{j=2}^k -2\log R_j. \tag{3.10b}
$$

The statistic $-2\log R_j$ is approximately chi square distributed with two degrees of freedom. Similarly $-2\log Q_k$ is approximately chi square distributed with $2(k-1)$ degrees of freedom. Readers should satisfy themselves that these numbers are indeed the correct, taking into account that each measurement $c_i$ has two free parameters $|S^a_{vv}|^2$ and $|S^b_{vh}|^2$, see [Eq. (2.13)](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#bivariate_change_detection).

Now for the algorithm:

### **The sequential omnibus change detection algorithm**



With a time series of $k$ SAR images $(c_1,c_2,\dots,c_k)$,

1. Set $\ell = k$.
2. Set $s = (c_{k-\ell+1}, \dots c_k)$.
3. Perform the omnibus test $Q_\ell$ for any changes change over $s$.
4. If no significant changes are found, stop.
5. Successively test series $s$ with $R_2, R_3, \dots$ until the first significant change is met for $R_j$.
6. Set $\ell = k-j+1$ and go to 2.

|Table 3.1 |       |       |       |       |       |        |
|----------|-------|-------|-------|-------|-------|--------|
|  $\ell$  | $c_1$ | $c_2$ | $c_3$ | $c_4$ | $c_5$ |        |
| 5        |       | $R^5_2$ | $R^5_3$ | $R^5_4$ | $R^5_5$ | $Q_5$  |
| 4        |       |       | $R^4_2$ | $R^4_3$ | $R^4_4$ | $Q_4$  |
| 3        |       |       |       | $R^3_2$ | $R^3_3$ | $Q_3$  |
| 2        |       |       |       |       | $R^2_2$ | $Q_2$  |


Thus if a change is found, the series is truncated up to the point of change and the testing procedure is repeated for the rest of the series. Take for example a series of $k=5$ images. (See Table 3.1 where, to avoid ambiguity, we add superscript $\ell$ to each $R_j$ test). Suppose there is one change in the second interval only. Then the test sequence is (the asterisk means $H_0$ is rejected)

$$
Q^*_5 \to R^5_2 \to R^{5*}_3 \to Q_3.
$$

If there are changes in the second and last intervals,

$$
Q^*_5 \to R^5_2 \to R^{5*}_3 \to Q^*_3 \to R^3_2 \to R^{3*}_3,
$$

and if there are significant changes in all four intervals,

$$
Q^*_5 \to R^{5*}_2 \to Q^*_4 \to R^{4*}_2 \to Q^*_3 \to R^{3*}_2 \to Q^*_2.
$$

The approach taken in the coding of this algorithm is to pre-calculate  _P_ values for all of the $Q_\ell / R_j$ tests and then, in a second pass, to filter them to determine the points of change.



### Pre-calculating the _P_ value array


The following code cell performs map operations on the indices $\ell$ and $j$, returning an array of _P_ values for all possible LRT statistics. For example again for $k=5$, the code calculates the _P_ values for each $R_j$ as a list of lists. Before calculating each row, the time series $c_1, c_2,c_3,c_4, c_5$ is sliced from $k-\ell+1$ to $k$. The last entry in each row is simply the product of the other entries,  $Q_\ell =\prod_{j=2}^\ell R_j.$

The program actually operates on the logarithms of the test statistics, Equations (3.10) (or 5.11 in the thesis paper).


In [ ]:
def det(im):
    """Calculates determinant of 2x2 diagonal covariance matrix."""
    return im.expression('b(0)*b(1)')

def log_det_sum(im_list, j):
    """Returns log of determinant of the sum of the first j images in im_list."""
    im_list = ee.List(im_list)
    sumj = ee.ImageCollection(im_list.slice(0, j)).reduce(ee.Reducer.sum())
    return ee.Image(det(sumj)).log()

def log_det(im_list, j):
    """Returns log of the determinant of the jth image in im_list."""
    im = ee.Image(ee.List(im_list).get(j.subtract(1)))
    return ee.Image(det(im)).log()

def pval(im_list, j, m=4.4):
    """Calculates -2logRj for im_list and returns P value and -2logRj."""
    im_list = ee.List(im_list)
    j = ee.Number(j)
    m2logRj = (log_det_sum(im_list, j.subtract(1))
               .multiply(j.subtract(1))
               .add(log_det(im_list, j))
               .add(ee.Number(2).multiply(j).multiply(j.log()))
               .subtract(ee.Number(2).multiply(j.subtract(1))
               .multiply(j.subtract(1).log()))
               .subtract(log_det_sum(im_list,j).multiply(j))
               .multiply(-2).multiply(m))
    pv = ee.Image.constant(1).subtract(chi2cdf(m2logRj, 2))
    return (pv, m2logRj)

def p_values(im_list):
    """Pre-calculates the P-value array for a list of images."""
    im_list = ee.List(im_list)
    k = im_list.length()

    def ells_map(ell):
        """Arranges calculation of pval for combinations of k and j."""
        ell = ee.Number(ell)
        # Slice the series from k-l+1 to k (image indices start from 0).
        im_list_ell = im_list.slice(k.subtract(ell), k)

        def js_map(j):
            """Applies pval calculation for combinations of k and j."""
            j = ee.Number(j)
            pv1, m2logRj1 = pval(im_list_ell, j)
            return ee.Feature(None, {'pv': pv1, 'm2logRj': m2logRj1})

        # Map over j=2,3,...,l.
        js = ee.List.sequence(2, ell)
        pv_m2logRj = ee.FeatureCollection(js.map(js_map))

        # Calculate m2logQl from collection of m2logRj images.
        m2logQl = ee.ImageCollection(pv_m2logRj.aggregate_array('m2logRj')).sum()
        pvQl = ee.Image.constant(1).subtract(chi2cdf(m2logQl, ell.subtract(1).multiply(2)))
        pvs = ee.List(pv_m2logRj.aggregate_array('pv')).add(pvQl)
        return pvs

    # Map over l = k to 2.
    ells = ee.List.sequence(k, 2, -1)
    pv_arr = ells.map(ells_map)

    # Return the P value array  ell = k,...,2, j = 2,...,l.
    return pv_arr
# Example call and debug for p_values
pv_arr = p_values(im_list)
pv_arr

### Filtering the _P_ values


|Table 3.2 |       |       |       |       |       |        |
|----------|-------|-------|-------|-------|-------|--------|
|$i\ $ / $j$|      |     1 |     2 |     3 |     4 |        |
| 1        |       | $P_2$ | $P_3$ | $P_4$ | $P_5$ | $P_{Q5}$  |
| 2        |       |       | $P_2$ | $P_3$ | $P_4$ | $P_{Q4}$  |
| 3        |       |       |       | $P_2$ | $P_3$ | $P_{Q3}$  |
| 4        |       |       |       |       | $P_2$ | $P_{Q2}$  |

The pre-calculated _P_ values in _pv\_arr_ (shown schematically in Table 3.2 for $k=5$) are then scanned in nested iterations over indices $i$ and $j$ to determine the following thematic change maps:

- cmap: the interval of the most recent change, one band, byte values $\in [0,k-1]$,
- smap: the interval of the first change, one band, byte values $\in [0,k-1]$,
- fmap: the number of changes, one band, byte values $\in [0,k-1]$,
- bmap: the changes in each interval, $\ k-1$ bands, byte values $\in [0,1]$).

A boolean variable _median_ is included in the code. Its purpose is to reduce the salt-and-pepper effect in no-change regions, which is at least partly a consequence of the uniform distribution of the _P_ values under $H_0$ (see the section [A note on P values](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#a_note_on_p_values) in Part 2). If _median_ is _True_, the _P_ values for each $Q_\ell$ statistic are passed through a $5\times 5$ median filter before being compared with the significance threshold. This is not statistically kosher but probably justifiable if one is only interested in large homogeneous changes, for example flood inundations or deforestation.

Here is the code:

In [ ]:
def filter_j(current, prev):
    """Calculates change maps; iterates over j indices of pv_arr."""
    pv = ee.Image(current)
    prev = ee.Dictionary(prev)
    pvQ = ee.Image(prev.get('pvQ'))
    i = ee.Number(prev.get('i'))
    cmap = ee.Image(prev.get('cmap'))
    smap = ee.Image(prev.get('smap'))
    fmap = ee.Image(prev.get('fmap'))
    bmap = ee.Image(prev.get('bmap'))
    alpha = ee.Image(prev.get('alpha'))
    j = ee.Number(prev.get('j'))
    cmapj = cmap.multiply(0).add(i.add(j).subtract(1))
    # Check      Rj?            Ql?                  Row i?
    tst = pv.lt(alpha).And(pvQ.lt(alpha)).And(cmap.eq(i.subtract(1)))
    # Then update cmap...
    cmap = cmap.where(tst, cmapj)
    # ...and fmap...
    fmap = fmap.where(tst, fmap.add(1))
    # ...and smap only if in first row.
    smap = ee.Algorithms.If(i.eq(1), smap.where(tst, cmapj), smap)
    # Create bmap band and add it to bmap image.
    idx = i.add(j).subtract(2)
    tmp = bmap.select(idx)
    bname = bmap.bandNames().get(idx)
    tmp = tmp.where(tst, 1)
    tmp = tmp.rename([bname])
    bmap = bmap.addBands(tmp, [bname], True)
    return ee.Dictionary({'i': i, 'j': j.add(1), 'alpha': alpha, 'pvQ': pvQ,
                          'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap':bmap})

def filter_i(current, prev):
    """Arranges calculation of change maps; iterates over row-indices of pv_arr."""
    current = ee.List(current)
    pvs = current.slice(0, -1 )
    pvQ = ee.Image(current.get(-1))
    prev = ee.Dictionary(prev)
    i = ee.Number(prev.get('i'))
    alpha = ee.Image(prev.get('alpha'))
    median = prev.get('median')
    # Filter Ql p value if desired.
    pvQ = ee.Algorithms.If(median, pvQ.focalMedian(2.5), pvQ)
    cmap = prev.get('cmap')
    smap = prev.get('smap')
    fmap = prev.get('fmap')
    bmap = prev.get('bmap')
    first = ee.Dictionary({'i': i, 'j': 1, 'alpha': alpha ,'pvQ': pvQ,
                           'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap': bmap})
    result = ee.Dictionary(ee.List(pvs).iterate(filter_j, first))
    return ee.Dictionary({'i': i.add(1), 'alpha': alpha, 'median': median,
                          'cmap': result.get('cmap'), 'smap': result.get('smap'),
                          'fmap': result.get('fmap'), 'bmap': result.get('bmap')})

The following function ties the two steps together:

In [ ]:
def change_maps(im_list, median=False, alpha=0.01):
    """Calculates thematic change maps."""
    k = im_list.length()
    # Pre-calculate the P value array.
    pv_arr = ee.List(p_values(im_list))
    # Filter P values for change maps.
    cmap = ee.Image(im_list.get(0)).select(0).multiply(0)
    bmap = ee.Image.constant(ee.List.repeat(0, k.subtract(1))).add(cmap)
    alpha = ee.Image.constant(alpha)
    first = ee.Dictionary({'i': 1, 'alpha': alpha, 'median': median,
                           'cmap': cmap, 'smap': cmap, 'fmap': cmap, 'bmap': bmap})
    return ee.Dictionary(pv_arr.iterate(filter_i, first))


## **Final Maps**

And now we run the algorithm , for our `im_list` which contains both polarization, for the level of significance.

We display the color-coded change maps: cmap, smap (blue early, red late) and fmap (blue few, red many) that we can clip (or not) to our buildings footprints:

In [ ]:
result = change_maps(im_list, median=True, alpha=0.01)
# Extract the change maps and display
# CLIP DESTROYED FOOTPRINTS
cmap = ee.Image(result.get('cmap'))#.clip(OSM_D_buf20)
smap = ee.Image(result.get('smap'))#.clip(OSM_D_buf20)
fmap = ee.Image(result.get('fmap'))#.clip(OSM_D_buf20)
bmap = ee.Image(result.get('bmap'))#.clip(OSM_D_buf20)

# CLIP NON-Destroyed FOOTPRINTS
cmap2 = ee.Image(result.get('cmap'))#.clip(OSM_ND_buf20)
smap2 = ee.Image(result.get('smap'))#.clip(OSM_ND_buf20)
fmap2 = ee.Image(result.get('fmap'))#.clip(OSM_ND_buf20)
bmap2 = ee.Image(result.get('bmap'))#.clip(OSM_ND_buf20)

# Define location and visualization palette
location = center.centroid().coordinates().getInfo()[::-1]
palette = ['yellow','orange', 'red'  ]

# Initialize the map
mp = geemap.Map(center=location, zoom=11)

# Mask out black (value = 0) pixels from each layer
cmap_masked = cmap.updateMask(cmap.neq(0))
smap_masked = smap.updateMask(smap.neq(0))
fmap_masked = fmap.updateMask(fmap.neq(0))

# Create a mask for black pixels (value = 0)
black_mask = cmap.eq(0)

# Add the black mask with reduced opacity
mp.addLayer(black_mask.updateMask(black_mask), {'palette': ['black'], 'opacity': 0.3}, 'Black Opacity')

# Add the layers to the map
mp.addLayer(cmap_masked, {'min': 0, 'max': 25, 'palette': palette}, 'cmap')
mp.addLayer(smap_masked, {'min': 0, 'max': 25, 'palette': palette}, 'smap')
mp.addLayer(fmap_masked, {'min': 0, 'max': 25, 'palette': palette}, 'fmap')

# Add vector layers with thicker lines
mp.addLayer(OSM_D_buf20.style(color='green', width=2, fillColor='00000000'), {}, 'OSM Destroyed')
mp.addLayer(OSM_ND_buf20.style(color='purple', width=2, fillColor='00000000'), {}, 'OSM Non-Destroyed')

# Add layer control
mp.addLayerControl()

# Display the map
mp



Map(center=[37.0662407844052, 37.38167089236449], controls=(WidgetControl(options=['position', 'transparent_bg…

### Interval Map

In [ ]:
# Definition of buffers and associated layers
buffers = {
    'OSM_D_buf20': {'geometry': OSM_D_buf20, 'smap': smap, 'bmap': bmap},
    'OSM_ND_buf20': {'geometry': OSM_ND_buf20, 'smap': smap2, 'bmap': bmap2}
}

# Visualization parameters with black background
visu_param = {'min': 0, 'max': 1, 'palette': ['000000', 'FF0000']}  # Original palette: Black for min, Red for max
black_visu_param = {'palette': ['black'], 'opacity': 0.3}  # Black opacity layer
mp = geemap.Map(center=location, zoom=11)

# Dictionary to store results for later use
results = {}

for buffer_name, buffer_info in buffers.items():
    buffer_geom = buffer_info['geometry']
    buffer_smap = buffer_info['smap']

    # Measure number of buildings with changes from the interval 12th
    interval_mask = buffer_smap.eq(11)
    interval_mask_int = interval_mask.toInt()

    # Mask out black (value = 0) pixels from each layer
    interval_mask_opacity = interval_mask.updateMask(interval_mask.neq(0))  # Mask black pixels

    # Add the masked layer with visualization parameters and reduced opacity
    mp.addLayer(interval_mask_opacity, {**visu_param, 'opacity': 0.5}, f'{buffer_name} Changes at Interval 12')

# Add the black mask layer for visualization with reduced opacity
black_mask = smap.eq(0)  # Create a mask for black pixels
mp.addLayer(black_mask.updateMask(black_mask), black_visu_param, 'Black Opacity')

# Add vector layers with thicker lines and no fill (empty)
mp.addLayer(OSM_D_buf20.style(color='green', width=2, fillColor='00000000'), {}, 'OSM Destroyed')
mp.addLayer(OSM_ND_buf20.style(color='purple', width=2, fillColor='00000000'), {}, 'OSM Non-Destroyed')

# Add layer control
mp.addLayerControl()

# Display the map
mp




#### Statistiques

In [ ]:
""" Number of buildings detected """
for buffer_name, buffer_info in buffers.items():
    buffer_geom = buffer_info['geometry']
    buffer_smap = buffer_info['smap']

    # Measure number of buildings with changes from the interval 12th
    interval_mask = buffer_smap.eq(11)
    interval_mask_int = interval_mask.toInt()

    # Compute the sum of change pixels within each building polygon
    stats = interval_mask_int.reduceRegions(
        collection=buffer_geom,
        reducer=ee.Reducer.sum(),
        scale=5
    )
    buildings_with_changes = stats.filter(ee.Filter.gt('sum', 0))
    number_of_buildings_with_changes = buildings_with_changes.size().getInfo()
    print(f'Number of buildings with changes in {buffer_name}: {number_of_buildings_with_changes}')

    # Observe values in cmap for each buffer
    stats1 = buffer_smap.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=buffer_geom.geometry(),
        scale=5,
        maxPixels=1e9
    )

    unique_values = stats1.getInfo()
    print(f"Unique values in smap for {buffer_name}: ", unique_values)

####Bmap

In [ ]:
# Define a custom reducer that computes both the sum and count of pixels
custom_reducer = ee.Reducer.sum().combine(
    reducer2=ee.Reducer.count(),
    sharedInputs=True
)
for buffer_name, buffer_info in buffers.items():
    buffer_geom = buffer_info['geometry']
    buffer_bmap = buffer_info['bmap']

    # Select the band corresponding to the 11th interval from bmap (index 10)
    bmap_interval_11 = buffer_bmap.select(11)

    # Convert interval to integer
    interval_mask_int = bmap_interval_11.toInt()

    # Apply the custom reducer to calculate the sum of change pixels and the total pixel count
    stats = interval_mask_int.reduceRegions(
        collection=buffer_geom,
        reducer=custom_reducer,
        scale=5
    )

    # Filter buildings with changes (sum > 0)
    buildings_with_changes = stats.filter(ee.Filter.gt('sum', 0))
    number_of_buildings_with_changes = buildings_with_changes.size().getInfo()
    print(f'Number of buildings with changes in {buffer_name} (bmap interval 11): {number_of_buildings_with_changes}')

    # Sum up the total number of change pixels and total pixels across all polygons
    total_change_pixels = stats.aggregate_sum('sum').getInfo()
    total_pixels = stats.aggregate_sum('count').getInfo()
    print(f"Total number of change pixels in {buffer_name} (interval 11): {total_change_pixels}")
    print(f"Total number of pixels in {buffer_name} (interval 11): {total_pixels}")


In [ ]:
def interval_stats(buffer_geom, map_images, interval_index):
    """Analyzes the specified interval for smap and bmap within the given buffer using a histogram."""

    results = {}

    for map_type, map_image in map_images.items():
        # Create the mask directly based on map_type
        interval_mask = (map_image.eq(interval_index) if map_type == 'smap'
                         else map_image.select(interval_index - 1).eq(1))
        # Apply the frequency histogram reducer to get a count of all pixel values
        pixel_histogram = interval_mask.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=buffer_geom,
            scale=5,
            maxPixels=1e9
        ).getInfo()

        # Number of buildings with at least one change pixel
        change_buildings = interval_mask.reduceRegions(
            collection=buffer_geom,
            reducer=ee.Reducer.sum(),
            scale=5
        ).filter(ee.Filter.gt('sum', 0)).size().getInfo()

        # Number of buildings without any change pixel
        total_buildings = buffer_geom.size().getInfo()
        no_change_buildings = total_buildings - change_buildings

        # Store the results in a dictionary
        results[map_type] = {
            'pixel_histogram': pixel_histogram,
            'change_buildings': change_buildings,}
    return results

In [ ]:
buffers = {
    'OSM_D_buf20': {'geometry': OSM_D_buf20, 'smap': smap, 'bmap': bmap},
    'OSM_ND_buf20': {'geometry': OSM_ND_buf20, 'smap': smap2, 'bmap': bmap2}
}

# Specify the interval index to analyze
interval_index = 10
# Initialize a dictionary to store results temporarily
by_type = {'smap': {}, 'bmap': {}}
# Perform analysis for each buffer
for buffer_name, buffer_info in buffers.items():
    buffer_geom = buffer_info['geometry']
    map_images = {'smap': buffer_info['smap'], 'bmap': buffer_info['bmap']}

    # Perform analysis for the given interval
    results = interval_stats(buffer_geom, map_images, interval_index)

    # Store the results organized by map type and buffer name
    for map_type in map_images.keys():
        by_type[map_type][buffer_name] = results[map_type]

# Output results for smap across all buffers first, then for bmap
for map_type in ['smap', 'bmap']:
    print(f'Results for {map_type}:')
    for buffer_name in buffers.keys():
        print(f'Buffer: {buffer_name}')
        print(f'Pixel Value Histogram in Interval {interval_index}: {by_type[map_type][buffer_name]["pixel_histogram"]}')
        print(f'Buildings with Change in Interval {interval_index}: {by_type[map_type][buffer_name]["change_buildings"]}')
    print('---')

